In [4]:
%run 3.inference.ipynb

Dataset salvo em ../datasets/edges.csv
     src   dest       rssi       etx      delay  busy_fraction  label
0  48045  51508 -63.000063  5.562383  96.908186       0.917675      0
1  38762  13616 -39.788324  3.145204  28.850376       0.107338      1
2  22127   2477 -89.612388  7.256131  90.952547       0.726642      0
3  20793  85827 -51.127332  3.533531  30.499648       0.023896      1
4  51281  77850 -79.060314  3.374069  60.044537       0.724193      0

Distribuição dos labels:
label
0    100000
1    100000
Name: count, dtype: int64


Epoch 0, Loss 0.6777, Train Acc 0.5946, Test Acc 0.7314
Epoch 10, Loss 0.3181, Train Acc 0.9819, Test Acc 0.9818
Epoch 20, Loss 0.1160, Train Acc 0.9813, Test Acc 0.9810
Epoch 30, Loss 0.0606, Train Acc 0.9825, Test Acc 0.9824
Epoch 40, Loss 0.0455, Train Acc 0.9845, Test Acc 0.9843

Acurácia final no conjunto de teste: 0.9861

Matriz de Confusão:
[[29419   581]
 [  252 29748]]

Relatório de Classificação:
              precision    recall  f1-score   s

In [5]:
import numpy as np

# ====== Probabilidade do GNN ======
prob_ml = probs[:, 1].numpy()

# Extraindo as probabilidades que o modelo atribui à classe "Bom"
# "Confiança" do modelo, não apenas a decisão binária.

# ====== Métricas Radnet ======
n = len(test_data)

APMR_edge = np.clip(0.65 + 0.20*np.sin(np.linspace(0, 3*np.pi, n)), 0, 1)
MFSR_edge = np.clip(0.70 + 0.15*np.cos(np.linspace(0, 2*np.pi, n)), 0, 1)
IGD_global = 0.62
IGD_vec = np.full(n, max(0.0, min(1.0, float(IGD_global))))

# APMR_edge (Active Prefix Message Ratio)
#   Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das
# arestas, medindo quão bem os campos de prefixo conseguem realizar matching
# probabilístico para encaminhar mensagens entre nós vizinhos.
# 
# MFSR_edge (Message Forwarding Success Ratio)
#   Proporção de mensagens transmitidas com sucesso através de uma aresta específica,
# considerando interferência, congestionamento e qualidade da conexão sem fio entre dois
# nós conectados.
# 
# IGD_global (Interest Group Delivery)
#   Eficácia global da entrega de mensagens no modo Interest-Group addressing, medindo
# quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham
# o mesmo interesse da aplicação.

# ====== Fusão espacial (ponderada) ======
w_ml, w_ap, w_mf, w_ig = 0.50, 0.20, 0.15, 0.15
final_score = (w_ml*prob_ml + w_ap*APMR_edge + w_mf*MFSR_edge + w_ig*IGD_vec)

# 50% da predição do GNN 
# 20% da métrica APMR 
# 15% da métrica MFSR 
# 15% da métrica IGD 

# ====== Decisão final ======
tau = 0.5

# threshold (limiar de decisão) para classificação binária.s
# Se final_score >= 0.5 → Classe 1 ("Bom")
# Se final_score < 0.5 → Classe 0 ("Ruim")

final_class = (final_score >= tau).astype(int)
CLASS_NAMES = {0: "Ruim", 1: "Bom"}

# ====== Resultados ======
print("\nSaída após fusão espacial (GNN + APMR_edge + MFSR_edge + IGD_global):")

for i, row in enumerate(test_data):
    print(
        f"Aresta {row} | "
        f"MLP={prob_ml[i]:.2f}, APMR_edge={APMR_edge[i]:.2f}, MFSR_edge={MFSR_edge[i]:.2f}, IGD={IGD_vec[i]:.2f} "
        f"-> Score={final_score[i]:.2f} => Classe {final_class[i]} ({CLASS_NAMES[final_class[i]]})"
    )

fusion_scores = final_score.tolist()


Saída após fusão espacial (GNN + APMR_edge + MFSR_edge + IGD_global):
Aresta [-55, 2, 17, 0.3] | MLP=0.99, APMR_edge=0.65, MFSR_edge=0.85, IGD=0.62 -> Score=0.85 => Classe 1 (Bom)
Aresta [-85, 3.8, 30, 0.1] | MLP=0.57, APMR_edge=0.68, MFSR_edge=0.85, IGD=0.62 -> Score=0.64 => Classe 1 (Bom)
Aresta [-35, 1.2, 5, 0.05] | MLP=1.00, APMR_edge=0.70, MFSR_edge=0.85, IGD=0.62 -> Score=0.86 => Classe 1 (Bom)
Aresta [-88, 9.5, 95, 0.9] | MLP=0.00, APMR_edge=0.73, MFSR_edge=0.84, IGD=0.62 -> Score=0.37 => Classe 0 (Ruim)
Aresta [-40, 8.2, 85, 0.8] | MLP=0.00, APMR_edge=0.75, MFSR_edge=0.84, IGD=0.62 -> Score=0.37 => Classe 0 (Ruim)
Aresta [-87, 1.5, 8, 0.15] | MLP=0.98, APMR_edge=0.78, MFSR_edge=0.83, IGD=0.62 -> Score=0.86 => Classe 1 (Bom)
Aresta [-62, 2.8, 32, 0.28] | MLP=0.95, APMR_edge=0.80, MFSR_edge=0.83, IGD=0.62 -> Score=0.85 => Classe 1 (Bom)
Aresta [-58, 3.2, 28, 0.32] | MLP=0.95, APMR_edge=0.81, MFSR_edge=0.82, IGD=0.62 -> Score=0.86 => Classe 1 (Bom)
Aresta [-52, 2.1, 20, 0.25] | M